# Project Description

The coronavirus took the entire world by surprise, changing everyone's daily routine. City dwellers no longer spent their free time outside, going to cafes and malls; more people were home, reading books. That attracted the attention of startups that rushed to develop new apps for book lovers.  
    
    
This study's aim to generate a value proposition for a new product using a database of one of the services competing in this market which contains data on books, publishers, authors, and customer ratings and reviews of books.  
    
In the project we will be working with 5 datasets :
- **`books`** : Which contains the following information on books : book_id, author_id, title, num_pages (number of pages), publication_date and publisher_id ;
- **`authors`** : Which contains informations on authors : author_id and author ;
- **`publishers`** : Which contains data on publishers : publisher_id and publisher ; 
- **`ratings`** : Which contains data on user ratings : rating_id, book_id, username (the name of the user who rated the book) and rating ;
- **`reviews`** : Which contains data on customer reviews : review_id, book_id, username and text (the text of the review).

# Table of contents  

1. [Data Preprocessing and Exploratory Analysis](#1)   
    1.1. [Importing and Examining Data](#11)    
    1.2. [Outliers and Correlation](#12)   
    1.3. [Client Study](#13) 
2. [Machine Learning and Predictions](#2)    
3. [User Categorization](#3)     
    3.1. [Hierarchical Clustering](#31)    
    3.2. [K-Means Clustering](#32)   
    3.3. [Cluster Visualization](#33)     
4. [Conclusion](#4) 
   

## 1.Examining Datasets <a name="1"></a>

**In this first phase, we will retreive the datasets we will be working with.**

In [1]:
!pip install seaborn --upgrade

In [2]:
# Import libraries
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns

from sqlalchemy import create_engine


db_config = {'user': 'practicum_student',         # username
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': 'data-analyst-final-project-db'}          # the name of the database

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

ModuleNotFoundError: No module named 'psycopg2'

In [ ]:
# Retrieving datasets
Q_books = '''SELECT * FROM books '''
Q_authors = '''SELECT * FROM authors '''
Q_publishers = '''SELECT * FROM publishers'''
Q_ratings = '''SELECT * FROM ratings '''
Q_reviews = '''SELECT * FROM reviews '''
        
books = pd.io.sql.read_sql(Q_books, con = engine)
authors = pd.io.sql.read_sql(Q_authors, con = engine)
publishers = pd.io.sql.read_sql(Q_publishers, con = engine)
ratings = pd.io.sql.read_sql(Q_ratings, con = engine)
reviews = pd.io.sql.read_sql(Q_reviews, con = engine)

# Displaying heads of the datasets
print('-------------------------------------------- Books Dataset --------------------------------------------')
display(books.head())
print('------------------------------------------- Authors Dataset -------------------------------------------')
display(authors.head())
print('----------------------------------------- Publishers Dataset ------------------------------------------')
display(publishers.head())
print('------------------------------------------- Ratings Dataset -------------------------------------------')
display(ratings.head())
print('------------------------------------------- Reviews Dataset -------------------------------------------')
display(reviews.head())

# Infos about the datatsets
print('------------------------------------- Books Dataset Informations --------------------------------------')
display(books.info())
print('------------------------------------ Authors Dataset Informations -------------------------------------')
display(authors.info())
print('----------------------------------- Publishers Dataset Informations -----------------------------------')
display(publishers.info())
print('------------------------------------ Ratings Dataset Informations -------------------------------------')
display(ratings.info())
print('------------------------------------ Reviews Dataset Informations -------------------------------------')
display(reviews.info())

**Observations :** 
    
From the above displays, we see that we are working with 5 datasets with no missing data. 

## 2. Data Analysis  <a name="2"></a>

### 2.1. Book Releases  <a name="21"></a>

We will calculate how many books were released after the 1st of January.

In [ ]:
Q1 = '''
        SELECT 
            COUNT (DISTINCT (book_id))
        FROM
            books
        WHERE
            publication_date > '2000-01-01'
     '''
book_count = pd.io.sql.read_sql(Q1, con = engine).iloc[0][0]

print('------------------------------------------------------------------------------------------')
print('')
print('The number of book released after the 1st of January 2000 is :', book_count, 'books')
print('')
print('------------------------------------------------------------------------------------------')

**Observations :**  

819 books were released after the 1st of January 2000, which makes for 81,9% of the total releases.


### 2.2. Reviews and Ratings  <a name="22"></a>

We will calculate the number of user reviews and the average rating for each book.

In [ ]:
Q2 = '''
        SELECT
            books.book_id,
            COUNT(reviews.username) as Reviews_count,
            ROUND(AVG(rating), 2) as Average_rating
            
        FROM
            books 
            LEFT JOIN reviews ON (books.book_id = reviews.book_id)
            LEFT JOIN ratings ON (books.book_id = ratings.book_id)
        GROUP BY
            books.book_id
     '''
reviews_ratings = pd.io.sql.read_sql(Q2, con = engine)

#Display
reviews_ratings.sort_values(by = 'book_id').head()


In [ ]:
# Dataset description
display(reviews_ratings.describe())

# Plot bar chart
sns.set_style("darkgrid")
sns.displot(reviews_ratings, x = 'average_rating', kde = True, alpha = .5, 
            color = '#b2182b',
            height = 5 , aspect = 2)
plt.title('Average Ratings Distribution')
plt.show() 

# Plot bar chart
sns.set_style("darkgrid")
sns.displot(reviews_ratings, x = 'reviews_count', kde = True, alpha = .5, bins = 50,
            height = 5 , aspect = 2)
plt.title('Reviews Count Distribution')
plt.show() 

**Observations :**  
    
We notice from the 'Average Ratings Distribution' graph above that **the majority of books are rated between 3,5 and 4,5 while there is a minority of books rated under 2,5**. For the 'Reviews Count Distribution', we notice that **the majority of books have no reviews while there are books that got more than 1120 reviews**.  
    
The mean average rating of the books is 3,9 and the averages number of reviews is 26 reviews. Also **the minimum average rating for a book is 1,5 while the maximum is 5**.

### 2.3. The Most Active Publisher  <a name="23"></a>

In [ ]:
Q3 = '''
        SELECT publishers.publisher, count(*) 
        FROM publishers
        LEFT JOIN "books" ON "publishers"."publisher_id" = "books"."publisher_id" 
        GROUP BY publishers.publisher_id
        HAVING COUNT(publishers.publisher_id) = (SELECT COUNT(publisher_id) 
                         FROM   books
                         WHERE num_pages > 50
                         GROUP  BY publisher_id
                         ORDER  BY COUNT(publisher_id) DESC 
                         LIMIT  1)  
     '''


best_publisher = pd.io.sql.read_sql(Q3, con = engine)

#Display
best_publisher


**Observations :**  
    
**The most active publisher with books exceeding 50 pages is 'Penguin Books' with 42 books published.**


### 2.4. Highest Rated Author  <a name="24"></a>

We will identify the author with the highest average book rating for books with more than 50 ratings.

In [ ]:
Q4 = '''
        SELECT books.title, authors.author, COUNT(*) , ROUND(AVG(ratings.rating),2)
        FROM books
        LEFT JOIN "authors" ON "authors"."author_id" = "books"."author_id" 
        LEFT JOIN "ratings" ON "books"."book_id" = "ratings"."book_id" 
        GROUP BY books.title, authors.author
        HAVING AVG(ratings.rating) = (SELECT AVG(ratings.rating) 
                         FROM books
                         LEFT JOIN "ratings" ON "books"."book_id" = "ratings"."book_id"
                         GROUP  BY books.book_id
                         HAVING COUNT(ratings.rating) > 50
                         ORDER  BY AVG(ratings.rating) DESC 
                         LIMIT  1)
                    
     '''


highest_rated_author = pd.io.sql.read_sql(Q4, con = engine)

#Display
highest_rated_author


**Observations :**  
    
The highest rated author with more than 50 ratings for a book is : **J.K. Rowling and Mary GrandPré with their book "Harry Potter and the Prisoner of Azkaban" which has averaged at 4.41 with 82 ratings.**

### 2.5. Average Number of Reviews  <a name="25"></a>

We will now find the average number of text reviews among users who rated more than 50 books.

In [ ]:
Q5 = '''
        SELECT ROUND(AVG(Count),2)
        FROM ( SELECT COUNT(reviews.review_id) as Count
                FROM reviews 
                GROUP BY reviews.username
                HAVING reviews.username IN (SELECT ratings.username
                                            FROM ratings
                                            GROUP BY ratings.username
                                            HAVING COUNT(ratings.rating_id) > 50 )
                ) as Counting
                    
     '''


average_reviews_count = pd.io.sql.read_sql(Q5, con = engine).iloc[0][0]

#Display
print('------------------------------------------------------------------------------------------')
print('')
print('The average number of reviews among users who rated more than 50 books is :', average_reviews_count, 'reviews per user.')
print('')
print('------------------------------------------------------------------------------------------')

In [ ]:
Q6 = '''
        SELECT ROUND(AVG(subq.Count),2)
        FROM ( SELECT 
                    COUNT(reviews.review_id) as Count
                FROM reviews 
                GROUP BY reviews.username
                having reviews.username IN(SELECT ratings.username
                                            FROM ratings
                                            GROUP BY ratings.username
                                            HAVING COUNT(ratings.rating_id) > 50 )
                ) as subq
                    
     '''


average_reviews_count = pd.io.sql.read_sql(Q6, con = engine).iloc[0][0]

#Display
print('------------------------------------------------------------------------------------------')
print('')
print('The average number of reviews among users who rated more than 50 books is :', average_reviews_count, 'reviews per user.')
print('')
print('------------------------------------------------------------------------------------------')

**Observations :**  
    
The average number of reviews among the active users who have rated more than 50 books is : **24,33 reviews per user.**